In [ ]:
!pip install selenium

In [1]:
# Importa el módulo webdriver de Selenium, que permite controlar el navegador
from selenium import webdriver

# Importa la clase By, que proporciona métodos para localizar elementos en la página web
from selenium.webdriver.common.by import By

# Importa la clase Select, que se usa para interactuar con menús desplegables en formularios HTML
from selenium.webdriver.support.ui import Select

# Importa la clase WebDriverWait, que permite implementar esperas explícitas
from selenium.webdriver.support.ui import WebDriverWait

# Importa el módulo expected_conditions con el alias EC, que contiene condiciones predefinidas para WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Importa la clase Service del módulo chrome.service y la renombra como ChromeService, para gestionar ChromeDriver
from selenium.webdriver.chrome.service import Service as ChromeService

# Importa la clase ChromeDriverManager del paquete webdriver_manager, para gestionar las versiones de ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager

import time

In [2]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')  # Ejecuta Chrome en modo headless 
# (es útil cuando ya estamos seguros de que el código sirve)

In [3]:
# creación de la instancia instancia del navegador web
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                          options=options)

In [4]:
def load_all_elements():
    """Carga todos los elementos desplazándose hacia abajo."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # Espera a que se carguen más elementos
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [5]:
def extract_information(bib):
    """Extrae la información de la página de resultados."""
    try:
        name = driver.find_element(By.ID, "nameCorredor").text
        tiempo_chip = driver.find_element(By.ID, "tdChip").text
        pace_per_km = driver.find_element(By.ID, "tdPaso").text
        average_speed = driver.find_element(By.ID, "tdPace").text
        
        splits = driver.find_element(By.ID, "raceResultRacePointColl")
        split_times = splits.find_elements(By.CLASS_NAME, "time-title")
        split_5k = split_times[0].text if len(split_times) > 0 else None
        split_10k = split_times[1].text if len(split_times) > 1 else None
        split_15k = split_times[2].text if len(split_times) > 2 else None
        split_20k = split_times[3].text if len(split_times) > 3 else None
        
        return {
            "BIB": bib,
            "Nombre": name,
            "Tiempo Chip": tiempo_chip,
            "Paso por Km": pace_per_km,
            "Velocidad Media": average_speed,
            "Split 5k": split_5k,
            "Split 10k": split_10k,
            "Split 15k": split_15k,
            "Split 20k": split_20k
        }
    except Exception as e:
        print(f"Error extrayendo información: {e}")
        return None

In [6]:
try:
    # Abre la página web
    driver.get("https://resultados.marcate.events/resultado/SPTMCAR1708026754")

    # Espera a que el dropdown esté presente
    wait = WebDriverWait(driver, 10) #Esto crea una instancia de WebDriverWait con un tiempo de 
    # espera máximo de 10 segundos antes de mandar una excepción
    
    # wait.until : el código debe esperar hasta que se cumpla una condición específica
    # EC.presence_of_element_located((By.ID, "categoria")) es la condición que estamos esperando
    # By.ID especifica que estamos buscando un elemento por su atributo id.
    # "categoria" es el valor del atributo id del elemento que estamos esperando.
    categoria_dropdown = wait.until(EC.presence_of_element_located((By.ID, "categoria")))
    #categoria_dropdown es un WebElement que contiene varias opciones selccionables
    
    # dormimos un rato el proceso para no levantar sospechas
    # Abre el dropdown y selecciona "LIBRE FEMENIL (18 A 34 AñOS)", lo hace creando una instancia
    # se Select que nos permite seleccionar objetos en menús por ejemplo. 
    # En este caso seleccionamos el que contiene el texto que nos interesa.
    select = Select(categoria_dropdown)
    select.select_by_visible_text("LIBRE FEMENIL (18 A 34 AñOS)")
    
    time.sleep(3)

    # Espera a que el botón de filtrar esté presente y haz clic en él
    # Lo hace creando una instancia de WebElement que representa el botón sobre el que haremos click
    filtrar_button = wait.until(EC.element_to_be_clickable((By.ID, "LastBtnfiltro")))
    filtrar_button.click() # hacemos click
    
    load_all_elements()
    
    # Identificamos que las tarjetas sobre las que queremos hacer click son 
    # de clase testimonial y las seleccionamos todas con presence_of_all_elements_located
    wait = WebDriverWait(driver, 10)
    elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "testimonial")))
    
    bib_numbers = []
    for element in elements:
        bib_text = element.find_element(By.XPATH, ".//li[strong[text()='BIB:']]").text
        try:
            bib_number = bib_text.split(":")[1].strip()
            bib_numbers.append(bib_number)
        except:
            pass
        
    # Genera y procesa cada URL usando los números de BIB
    results = []
    for bib_number in bib_numbers:
        try:
            # Genera la URL basada en el número de BIB
            result_url = f"https://resultados.marcate.events/resultado/SPTMCAR1708026754/{bib_number}"
            driver.get(result_url) # Abre esa página
            
            time.sleep(2)
            
            # Extrae la información de la página de resultados
            info = extract_information(bib_number)
            print(info)
            if info:
                results.append(info)
            
            time.sleep(2)
            
        except Exception as e:
            print(f"Error procesando el BIB {bib_number}: {e}")


finally:
    # Cierra el navegador
    # driver.quit()
    pass

{'BIB': '71', 'Nombre': 'GABRIELA FLORES GARCIA', 'Tiempo Chip': '01:18:25', 'Paso por Km': '03:41 /km', 'Velocidad Media': '16.14 km/h', 'Split 5k': '00:18:02', 'Split 10k': '00:36:32', 'Split 15k': '00:55:47', 'Split 20k': '01:14:33'}
{'BIB': '63', 'Nombre': 'BRENDA EUNICE FLORES MUÑOS', 'Tiempo Chip': '01:20:00', 'Paso por Km': '03:47 /km', 'Velocidad Media': '15.82 km/h', 'Split 5k': '00:18:31', 'Split 10k': '00:38:13', 'Split 15k': '00:57:46', 'Split 20k': '01:16:08'}
{'BIB': '56', 'Nombre': 'ISABEL GUADALUPE OROPEZA VAZQUEZ', 'Tiempo Chip': '01:20:43', 'Paso por Km': '03:47 /km', 'Velocidad Media': '15.68 km/h', 'Split 5k': '00:18:09', 'Split 10k': '00:37:46', 'Split 15k': '00:57:39', 'Split 20k': '01:16:29'}
{'BIB': '67', 'Nombre': 'SONIA HERNÁNDEZ HERNÁNDEZ', 'Tiempo Chip': '01:22:18', 'Paso por Km': '03:53 /km', 'Velocidad Media': '15.38 km/h', 'Split 5k': '00:19:24', 'Split 10k': '00:39:38', 'Split 15k': '00:59:37', 'Split 20k': '01:18:18'}
{'BIB': '58', 'Nombre': 'MARIA JILO

{'BIB': '27985', 'Nombre': 'CARMEN VERA BRAVO', 'Tiempo Chip': '01:30:07', 'Paso por Km': '04:15 /km', 'Velocidad Media': '14.04 km/h', 'Split 5k': '00:21:42', 'Split 10k': '00:43:12', 'Split 15k': '01:04:54', 'Split 20k': '01:25:53'}
{'BIB': '24894', 'Nombre': 'MARIANA CHAVERO BRUNET', 'Tiempo Chip': '01:29:57', 'Paso por Km': '04:13 /km', 'Velocidad Media': '14.07 km/h', 'Split 5k': '00:21:48', 'Split 10k': '00:43:36', 'Split 15k': '01:05:45', 'Split 20k': '01:25:53'}
{'BIB': '20309', 'Nombre': 'VANESSA FERMOSO SÁNCHEZ', 'Tiempo Chip': '01:30:36', 'Paso por Km': '04:15 /km', 'Velocidad Media': '13.97 km/h', 'Split 5k': '00:20:51', 'Split 10k': '00:43:42', 'Split 15k': '01:05:29', 'Split 20k': '01:26:27'}
{'BIB': '20612', 'Nombre': 'NADIA SALINAS MENDOZA', 'Tiempo Chip': '01:36:30', 'Paso por Km': '04:33 /km', 'Velocidad Media': '13.11 km/h', 'Split 5k': '00:21:51', 'Split 10k': '00:45:30', 'Split 15k': '01:09:22', 'Split 20k': '01:31:45'}
{'BIB': '5474', 'Nombre': 'CAMILA SALINAS CAB

{'BIB': '20943', 'Nombre': 'MONICA MAGANA PEREZ', 'Tiempo Chip': '01:30:27', 'Paso por Km': '04:15 /km', 'Velocidad Media': '13.99 km/h', 'Split 5k': '00:21:11', 'Split 10k': '00:42:51', 'Split 15k': '01:04:56', 'Split 20k': '01:26:03'}
{'BIB': '26225', 'Nombre': 'ANA LUCIA RAMIREZ RUIZ', 'Tiempo Chip': '01:30:31', 'Paso por Km': '04:15 /km', 'Velocidad Media': '13.98 km/h', 'Split 5k': '00:21:12', 'Split 10k': '00:42:52', 'Split 15k': '01:04:56', 'Split 20k': '01:26:02'}
{'BIB': '13319', 'Nombre': 'ELIKNAHY ALVAREZ RAMIREZ', 'Tiempo Chip': '01:36:27', 'Paso por Km': '04:33 /km', 'Velocidad Media': '13.12 km/h', 'Split 5k': '00:22:43', 'Split 10k': '00:45:58', 'Split 15k': '01:09:25', 'Split 20k': '01:31:44'}
{'BIB': '7365', 'Nombre': 'MARIA JOSE DOMINGUEZ DIAZ', 'Tiempo Chip': '01:37:17', 'Paso por Km': '04:35 /km', 'Velocidad Media': '13.01 km/h', 'Split 5k': '00:22:37', 'Split 10k': '00:46:10', 'Split 15k': '01:10:13', 'Split 20k': '01:32:34'}
{'BIB': '28213', 'Nombre': 'ISABEL OCAM

{'BIB': '27989', 'Nombre': 'ARANZA QUIROZ ROEL', 'Tiempo Chip': '01:33:45', 'Paso por Km': '04:24 /km', 'Velocidad Media': '13.50 km/h', 'Split 5k': '00:21:55', 'Split 10k': '00:44:31', 'Split 15k': '01:07:18', 'Split 20k': '01:29:07'}
{'BIB': '12330', 'Nombre': 'MAYRA TANIA PÉREZ ANGELES', 'Tiempo Chip': '01:38:43', 'Paso por Km': '04:38 /km', 'Velocidad Media': '12.82 km/h', 'Split 5k': '00:24:03', 'Split 10k': '00:47:40', 'Split 15k': '01:11:39', 'Split 20k': '01:34:05'}
{'BIB': '9357', 'Nombre': 'MARIA ALMENDRA ROMERO CABALLERO', 'Tiempo Chip': '01:34:06', 'Paso por Km': '04:27 /km', 'Velocidad Media': '13.45 km/h', 'Split 5k': '00:21:35', 'Split 10k': '00:44:18', 'Split 15k': '01:07:18', 'Split 20k': '01:29:28'}
{'BIB': '5490', 'Nombre': 'SAMANTHA SANCHEZ CHAVEZ', 'Tiempo Chip': '01:32:48', 'Paso por Km': '04:21 /km', 'Velocidad Media': '13.64 km/h', 'Split 5k': '00:22:17', 'Split 10k': '00:44:51', 'Split 15k': '01:07:02', 'Split 20k': '01:28:25'}
{'BIB': '3060', 'Nombre': 'LILIA 

{'BIB': '18227', 'Nombre': 'ELSA RIVERA URIBE', 'Tiempo Chip': '01:42:02', 'Paso por Km': '04:50 /km', 'Velocidad Media': '12.40 km/h', 'Split 5k': '00:24:18', 'Split 10k': '00:49:01', 'Split 15k': '01:13:45', 'Split 20k': '01:37:05'}
{'BIB': '1391', 'Nombre': 'NANCY FIERRO CABRALES', 'Tiempo Chip': '01:42:13', 'Paso por Km': '04:50 /km', 'Velocidad Media': '12.38 km/h', 'Split 5k': '00:24:04', 'Split 10k': '00:48:20', 'Split 15k': '01:13:08', 'Split 20k': '01:37:24'}
{'BIB': '14863', 'Nombre': 'ELISA FERNANDEZ OTERO', 'Tiempo Chip': '01:37:00', 'Paso por Km': '04:33 /km', 'Velocidad Media': '13.05 km/h', 'Split 5k': '00:22:42', 'Split 10k': '00:45:38', 'Split 15k': '01:08:56', 'Split 20k': '01:31:55'}
{'BIB': '14522', 'Nombre': 'FERNANDA AGUILERA GARCIA', 'Tiempo Chip': '01:36:42', 'Paso por Km': '04:33 /km', 'Velocidad Media': '13.09 km/h', 'Split 5k': '00:22:31', 'Split 10k': '00:46:23', 'Split 15k': '01:09:44', 'Split 20k': '01:32:01'}
{'BIB': '6289', 'Nombre': 'ROMINA HAIK COLO', 

...

{'BIB': '21908', 'Nombre': 'MARIA DEL CARMEN GALLARDO VALENCIA', 'Tiempo Chip': '03:06:37', 'Paso por Km': '08:48 /km', 'Velocidad Media': '6.782 km/h', 'Split 5k': '00:39:44', 'Split 10k': '01:21:42', 'Split 15k': '02:11:04', 'Split 20k': '02:58:15'}
{'BIB': '26697', 'Nombre': 'SARITA BLANCO CASTAÑEDA', 'Tiempo Chip': '03:05:28', 'Paso por Km': '08:46 /km', 'Velocidad Media': '6.825 km/h', 'Split 5k': '00:42:03', 'Split 10k': '01:22:52', 'Split 15k': '02:17:45', 'Split 20k': '02:57:26'}
{'BIB': '11771', 'Nombre': 'MARCELA MARIANA MARTINEZ FLORES', 'Tiempo Chip': '03:22:34', 'Paso por Km': '09:34 /km', 'Velocidad Media': '6.249 km/h', 'Split 5k': '00:41:43', 'Split 10k': '01:28:36', 'Split 15k': '02:19:46', 'Split 20k': '03:13:07'}
{'BIB': '16810', 'Nombre': 'PAOLA LANDA ORTEGA', 'Tiempo Chip': '03:03:24', 'Paso por Km': '08:40 /km', 'Velocidad Media': '6.901 km/h', 'Split 5k': '00:39:35', 'Split 10k': '01:22:11', 'Split 15k': '02:08:14', 'Split 20k': '02:54:38'}
{'BIB': '22725', 'Nomb

{'BIB': '27608', 'Nombre': 'VERONICA GUIZAR WADE', 'Tiempo Chip': '03:24:04', 'Paso por Km': '09:40 /km', 'Velocidad Media': '6.203 km/h', 'Split 5k': '00:42:49', 'Split 10k': '01:28:29', 'Split 15k': '02:20:36', 'Split 20k': '03:14:18'}
{'BIB': '19690', 'Nombre': 'ILIANA ROMO QUIROZ', 'Tiempo Chip': '03:11:40', 'Paso por Km': '09:03 /km', 'Velocidad Media': '6.604 km/h', 'Split 5k': '00:40:17', 'Split 10k': '01:24:08', 'Split 15k': '02:11:45', 'Split 20k': '03:02:06'}
{'BIB': '16134', 'Nombre': 'ANDREA NIELSEN LIZARRAGA', 'Tiempo Chip': '03:04:57', 'Paso por Km': '08:43 /km', 'Velocidad Media': '6.844 km/h', 'Split 5k': '00:42:12', 'Split 10k': '01:24:08', 'Split 15k': '02:12:02', 'Split 20k': '02:56:19'}
{'BIB': '8246', 'Nombre': 'ANA KAREN AHUMADA ANAYA', 'Tiempo Chip': '03:02:00', 'Paso por Km': '08:37 /km', 'Velocidad Media': '6.955 km/h', 'Split 5k': '00:38:02', 'Split 10k': '01:21:00', 'Split 15k': '02:08:08', 'Split 20k': '02:52:10'}
{'BIB': '14041', 'Nombre': 'VANESSA CHAPA CA

{'BIB': '8167', 'Nombre': 'KAREN ALINE PINEDA ZUNIGA', 'Tiempo Chip': '03:15:59', 'Paso por Km': '09:14 /km', 'Velocidad Media': '6.458 km/h', 'Split 5k': '00:42:42', 'Split 10k': '01:38:56', 'Split 15k': '02:22:24', 'Split 20k': '03:06:47'}
{'BIB': '3163', 'Nombre': 'ALEJANDRA RIVAS ALONSO', 'Tiempo Chip': '03:16:07', 'Paso por Km': '09:17 /km', 'Velocidad Media': '6.454 km/h', 'Split 5k': '00:38:32', 'Split 10k': '01:22:55', 'Split 15k': '02:13:14', 'Split 20k': '03:05:27'}
{'BIB': '19193', 'Nombre': 'TANIA AZUCENA GARCÍA GONZÁLEZ', 'Tiempo Chip': '03:10:43', 'Paso por Km': '09:00 /km', 'Velocidad Media': '6.637 km/h', 'Split 5k': '00:42:04', 'Split 10k': '01:27:21', 'Split 15k': '02:15:00', 'Split 20k': '03:01:37'}
{'BIB': '499', 'Nombre': 'BRENDA VELAZQUEZ DODGE', 'Tiempo Chip': '03:25:12', 'Paso por Km': '09:43 /km', 'Velocidad Media': '6.169 km/h', 'Split 5k': '00:40:04', 'Split 10k': '01:25:15', 'Split 15k': '02:18:18', 'Split 20k': '03:16:12'}
{'BIB': '6926', 'Nombre': 'ABRIL A

{'BIB': '8413', 'Nombre': 'ANA PAULA LOPEZ RIVERA', 'Tiempo Chip': '03:11:25', 'Paso por Km': '09:03 /km', 'Velocidad Media': '6.612 km/h', 'Split 5k': '00:42:19', 'Split 10k': '01:27:16', 'Split 15k': '02:14:45', 'Split 20k': '03:02:50'}
{'BIB': '9021', 'Nombre': 'ANDREA MORALES VELASCO', 'Tiempo Chip': '03:21:44', 'Paso por Km': '09:31 /km', 'Velocidad Media': '6.275 km/h', 'Split 5k': '00:44:00', 'Split 10k': '01:30:18', 'Split 15k': '02:20:08', 'Split 20k': '03:11:45'}
{'BIB': '29345', 'Nombre': 'LUZ ELENA ESPINOSA PEREZ', 'Tiempo Chip': '03:20:16', 'Paso por Km': '09:28 /km', 'Velocidad Media': '6.321 km/h', 'Split 5k': '00:41:06', 'Split 10k': '01:31:59', 'Split 15k': '02:17:59', 'Split 20k': '03:11:39'}
{'BIB': '21439', 'Nombre': 'MIRIAM SORIA LOPEZ', 'Tiempo Chip': '03:09:16', 'Paso por Km': '08:57 /km', 'Velocidad Media': '6.688 km/h', 'Split 5k': '00:41:36', 'Split 10k': '01:23:40', 'Split 15k': '02:09:56', 'Split 20k': '02:59:20'}
{'BIB': '24981', 'Nombre': 'REGINA GARCIA LA

{'BIB': '25453', 'Nombre': 'SONIA MIRÉN MARTÍNEZ GONZÁLEZ', 'Tiempo Chip': '03:26:59', 'Paso por Km': '09:45 /km', 'Velocidad Media': '6.115 km/h', 'Split 5k': '00:42:00', 'Split 10k': '01:27:34', 'Split 15k': '02:17:09', 'Split 20k': '03:13:26'}
{'BIB': '14717', 'Nombre': 'DARIENNI SANCHEZ JIMÉNEZ', 'Tiempo Chip': '03:27:28', 'Paso por Km': '09:48 /km', 'Velocidad Media': '6.101 km/h', 'Split 5k': '00:44:04', 'Split 10k': '01:30:36', 'Split 15k': '02:23:13', 'Split 20k': '03:17:57'}
{'BIB': '25002', 'Nombre': 'MARIA GUADALUPE CESAR MENDOZA', 'Tiempo Chip': '03:57:19', 'Paso por Km': '11:13 /km', 'Velocidad Media': '5.334 km/h', 'Split 5k': '00:56:37', 'Split 10k': '01:48:46', 'Split 15k': '02:45:20', 'Split 20k': '03:45:26'}
{'BIB': '25001', 'Nombre': 'PAULINA CESAR MENDOZA', 'Tiempo Chip': '03:57:19', 'Paso por Km': '11:13 /km', 'Velocidad Media': '5.334 km/h', 'Split 5k': '00:56:37', 'Split 10k': '01:48:47', 'Split 15k': '02:45:22', 'Split 20k': '03:45:26'}
{'BIB': '10920', 'Nombre'

In [7]:
len(results)

6186

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(results)

In [10]:
df

,BIB,Nombre,Tiempo Chip,Paso por Km,Velocidad Media,Split 5k,Split 10k,Split 15k,Split 20k
0,71,GABRIELA FLORES GARCIA,01:18:25,03:41 /km,16.14 km/h,00:18:02,00:36:32,00:55:47,01:14:33
1,63,BRENDA EUNICE FLORES MUÑOS,01:20:00,03:47 /km,15.82 km/h,00:18:31,00:38:13,00:57:46,01:16:08
2,56,ISABEL GUADALUPE OROPEZA VAZQUEZ,01:20:43,03:47 /km,15.68 km/h,00:18:09,00:37:46,00:57:39,01:16:29
3,67,SONIA HERNÁNDEZ HERNÁNDEZ,01:22:18,03:53 /km,15.38 km/h,00:19:24,00:39:38,00:59:37,01:18:18
4,58,MARIA JILOTE VELA,01:22:41,03:53 /km,15.31 km/h,00:19:09,00:39:29,00:59:45,01:18:39
...,...,...,...,...,...,...,...,...,...
6181,2085,SONIA FRANCISCO ATZIN,03:29:48,09:54 /km,6.033 km/h,00:42:28,01:30:30,02:25:38,03:19:05
6182,19252,OLINDA MADELEINE RODRIGUEZ PICAZO,03:33:36,10:05 /km,5.926 km/h,00:43:51,01:33:38,02:26:57,03:22:36
6183,12038,MARIAM OTERO PEÑA,03:16:38,09:17 /km,6.437 km/h,00:37:21,01:20:11,02:09:04,03:05:51
6184,10044,ARANZA GABRIELA SAAVEDRA SOTO,03:29:37,09:54 /km,6.038 km/h,00:44:39,01:31:55,02:23:37,03:18:35
